In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [3]:
from googleapiclient.discovery import build
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [ ]:
def prettify(x):
    if type(x) == str and ',' in x:
        x = list(x)
        x.remove(',')
        return ''.join(x)
    elif x == '':
        return 0.
    elif x == '-':
        return 0
    else:
        return x

In [4]:
import pandas as pd
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1DsdShP0FXDuOyrYXt-5U9tWDbDMRVdgvYxzQ24qHZic'
DATA_TO_PULL = 'DATA'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)    

COMPLETE: Data copied


In [70]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1IswzB83DTpBdFQ91Lfr-OrDfDiEi4CMQefjeNIemU0I'
DATA_TO_PULL = 'Backlog H MM only'
mm_coins = pd.DataFrame(pull_sheet_data(SCOPES, SPREADSHEET_ID, DATA_TO_PULL))[0]

COMPLETE: Data copied


In [280]:
df = pd.DataFrame(data)
coins = [i for i in df.iloc[0] if i != '' and i != None]
ind = pd.MultiIndex.from_product([coins, ['Fee', 'Volume', 'Count']])
df.drop(index=[0, 2], inplace=True)
df.drop(columns=[0, 1], inplace=True)
d = df.to_numpy()
dff = pd.DataFrame(d, columns=ind)
dff.fillna(0, inplace=True)
select = [i for i in ind if 'Fee' in i[1]]
dff = dff[select]

In [281]:
dff = dff.applymap(prettify).astype(float)
dff = dff.transpose().sort_values(by=0, ascending=False)
dff = dff.transpose()
# dff.drop(index=0, inplace=True)

In [245]:
mm_only = [i in coins for i in mm_coins]
mm = []
for i in range(len(mm_only)):
    if mm_only[i] == True:
        mm.append(mm_coins[i])
    else:
        continue

In [282]:
dff1 = dff.applymap(prettify)
idxs = range(len(dff)-4, 1, -4)
dff1 = dff1.iloc[idxs][:7].astype(np.float32)

In [287]:
temp = pd.DataFrame(data)
for_total1 = temp.iloc[1]
ft_index = list(range(2, len(for_total1), 3))
ft2 = for_total1[ft_index].apply(prettify).astype(float)
total = sum(ft2)

In [284]:
select = [i for i in ind if 'Fee' in i[1] and i[0] in mm]
total_mm = dff[select].iloc[0].sum()
total_wo_mm = total - total_mm

In [289]:
table = {coins[i]:dff1.transpose().iloc[i].sum(axis=0) for i in range(80)}
table = {k: table[k] for k in sorted(table, key=table.get, reverse=True)}
table_wo_mm = defaultdict()
table_mm = defaultdict()
for i in range(len(dff.columns)):
    if dff.columns[i][0] not in mm:
        table_wo_mm[dff.columns[i][0]] = '$' + str(dff[dff.columns[i]][0]) + ' + ' + '$' + str(table[dff.columns[i][0]])
    else:
        table_mm[dff.columns[i][0]] = '$' + str(dff[dff.columns[i]][0]) + ' + ' + '$' + str(table[dff.columns[i][0]])

In [290]:
data_mm = {'Symbol MM':list(table_mm.keys()), 'Total':list(table_mm.values())}
df_mm = pd.DataFrame.from_dict((data_mm)).set_index('Symbol MM')
df_mm

,Total
Symbol MM,
REN,$8813.0 + $72.0
CAKE,$946.0 + $43.0
BAL,$316.0 + $34.0
XVS,$169.0 + $27.0
LPT,$156.0 + $15.0
CELR,$122.0 + $4.0
UMA,$107.0 + $12.0
TORN,$84.0 + $0.0
POLS,$82.0 + $0.0


In [291]:
data_wo_mm = {'Symbol':list(table_wo_mm.keys()), 'Total':list(table_wo_mm.values())}
df_wo_mm = pd.DataFrame.from_dict((data_wo_mm)).set_index('Symbol')
df_wo_mm

,Total
Symbol,
SHIB,$51340.0 + $3409.0
ICP,$4795.0 + $678.0
C98,$3420.0 + $148.0
MINA,$3290.0 + $119.0
AR,$2654.0 + $247.0
...,...
BADGER,$0.0 + $30.0
AMP,$0.0 + $37.0
IMX,$0.0 + $18.0


In [292]:
print(f'TOTAL = {total}, TOTAL w/o MM = {total_wo_mm}, TOTAL MM = {total_mm}')

TOTAL = 86557.87234549799, TOTAL w/o MM = 75590.87234549799, TOTAL MM = 10967.0
